## Задание 1

#### В этом задании необходимо продемонстрировать занния SQL, решив задания любым доступным способом

Таблица persons - таблица с людьми, относящимися к миру кино:
- id — уникальный идентификатор человека;
- name — имя;
- year_of_birth — год рождения;
- year_of_death — год смерти;
- professions — список профессий.

Таблица titles - таблица с информацией о произведениях (фильмах, сериалах, шоу):
- id — уникальный идентификатор;
- type — тип произведения: фильм, сериал, короткометражка;
- popular_title — популярное название;
- original_title — оригинальное название;
- year_of_start — год выпуска/старта сериала;
- year_of_end — год окончания сериала;
- is_adult — есть ли у фильма ограничение 18+;
- run_time — длительность в минутах;
- genres — жанры, к которым относится произведение.

Таблица ratings - таблица с рейтингами от зрителей:
- title_id — оцениваемое произведение;
- avg_rating — средний рейтинг;
- vote_cnt — количество голосовавших.

Таблица episodes - таблица с эпиздами сериала:
- title_id — ссылка на t itle_id этого эпизода;
- parent_id — ссылка на title_id всего сериала;
- season — номер сезона;
- episode — номер эпизода.

Таблица principals — таблица с людьми, участвовавшими в создании произведений:
- title_id — id произведения;
- ordering — уникальный номер в рамках одного title_id ;
- person_id — id из таблицы persons;
- category — категория работы в рамках произведения;
- job — конкретная работа. Человек может быть автором произведения и в рамках фильма работать над адаптацией текста;
- characters — персонажи, которых играл человек.

Написать SQL-запрос, который выведет количество живых и умерших людей из таблицы persons. 
Если поле year_of_death пустое, вывести alive, иначе — dead.

In [ ]:
SELECT (case when year_of_death is null then 'alive'
             else 'dead' end) as status, -- Используем конструкцию case для определения "живых" и "умерших"
       count(1) as cnt -- Считаем их количество
  FROM persons
 GROUP BY status -- Группируем по статусу
 ORDER BY status, cnt -- Сортируем по статусу и количеству

Написать SQL-запрос, который выведет средний рейтинг каждого из жанров.

In [ ]:
SELECT genres, avg(r.avg_rating) AS avg_rating
  FROM titles AS t
  LEFT JOIN ratings AS r -- Объединяем таблицы по id фильмов
    ON t.id = r.title_id
 WHERE genres NOT LIKE '%,%' -- Фильтруем строки, где фильмы с одним жанром
 GROUP BY genres -- группируем по жанрам
 ORDER BY avg_rating DESC -- Сортируем по убыванию среднего рейтинга

Написать SQL-запрос, который выведет жанр и сумму голосов (vote_cnt из таблицы ratings) для каждого жанра.

In [ ]:
SELECT genres,
       sum(r.vote_cnt) as sum_vote
  FROM titles as t
    LEFT JOIN ratings as r -- Объединяем таблицы по id фильмов
        ON t.id = r.title_id
 WHERE genres not like '%,%' -- Фильтруем строки, где фильмы с одним жанром
 GROUP BY genres -- группируем по жанрам
 ORDER BY sum_vote desc -- Сортируем по убыванию среднего рейтинга

Написать SQL-запрос, который выведет имена людей (поле name из таблицы persons), которые судя по таблице principals участвовали в создании 10 самых высокооценённых произведений (топ 10 по рейтингу (avg_rating) и количеству набранных голосов из таблицы ratings).

In [ ]:
WITH the_best_id AS (
    SELECT title_id
      FROM ratings
     ORDER BY avg_rating DESC, vote_cnt DESC
     LIMIT 10
), -- создадим временную таблицу с топ-10 фильмами по среденму рейтингу
prin AS (
    SELECT person_id
      FROM principals
     WHERE title_id IN (SELECT title_id FROM the_best_id)
) -- найдем id всех создателей фильмов, которые участвавали в создании топ-10 лучших фильмов

SELECT name
  FROM persons
 WHERE id IN (SELECT person_id FROM prin)
 ORDER BY name; -- выведем имена этих создателей

Написать SQL-запрос, который выведет названия (name) сериалов, у которых первый эпизод первого сезона получил рейтинг 10.

In [ ]:
WITH cte AS (
        SELECT (SELECT t.original_title
                  FROM titles AS t
                 WHERE t.id = e.parent_id) AS name,
               (SELECT r.avg_rating
                  FROM ratings AS r
                 WHERE r.title_id = e.title_id) AS avg_rating
          FROM episodes AS e
         WHERE e.episode = 1 AND e.season = 1) -- создадим cte, в котором объединим 3 таблицы и найдем названия лучших фильмов

SELECT name
  FROM cte
 WHERE avg_rating = 10
   AND name IS NOT NULL
 ORDER BY name; -- выводим названия лучших сериалов первого сезона

Вывести оригинальное название (original_title), год начала (year_of_start) и количество эпизодов произведений (ep_cnt), у которых больше 500 серий, отсортировав их по убыванию количества серий.

In [ ]:
WITH ep_char as (SELECT parent_id,
                        count(1) as ep_cnt
                 FROM   episodes as ep
                 GROUP BY parent_id 
                 HAVING count(1) > 500
                 ORDER BY ep_cnt desc) -- создадим cte для отображения фильмов, у которых более 500 серий
SELECT original_title,
       year_of_start,
       ep_char.ep_cnt
  FROM titles join ep_char
       ON ep_char.parent_id = titles.id
 ORDER BY ep_cnt desc -- выводим название, год начала и количество эпизодов

Написать SQL-запрос с использованием CTE, который выведет:

— имя (name);

— сезон (season).

и далее для каждого персонажа и сезона с сортировкой по возрастанию количества эпизодов в сезоне, где он встречается:

— количество эпизодов в сезоне (ep_cnt);

— номер строки в текущей партиции (rn);

— ранг строки с пропусками (rank);

— ранг строки без пропусков (dense_rank);

— номер группы при делении окна на две группы NTILE (group_num);

— кумулятивное распределение CUME_DIST (cume_dist);

— ранг в процентах PERCENT_RANK (percent_rank).

Воспользуемся WINDOW FRAME для упрощения кода

In [ ]:
WITH cte as (SELECT ch.name,
                    substr(ep.episode_id, 1, 3) as season,
                    count(distinct ep.episode_id) as ep_cnt
             FROM   characters as ch
                 LEFT JOIN char_ep ce
                     ON ch.id = ce.character_id
                 LEFT JOIN episodes as ep
                     ON ce.episode_id = ep.id
             GROUP BY ch.name, substr(ep.episode_id, 1, 3) having count(ep.episode_id) > 5)
SELECT name,
       season,
       ep_cnt,
       row_number() OVER w as rn,
       rank() OVER w as rank,
       dense_rank() OVER w as dense_rank,
       ntile(2) OVER w as group_num,
       cume_dist() OVER w as cume_dist,
       percent_rank() OVER w as percent_rank
FROM   cte window w as (
PARTITION BY name
ORDER BY ep_cnt)

### Задание 2

#### В данном задании необходимо решить три задачи, представляющие собой варианты тестовых заданий на собеседовании

Таблица employees - информация о сотрудниках:

- employee_id - уникальный идентификатор сотрудника;
- name - имя сотрудника;
- manager_id - уникальный идентификатор руководителя сотрудника;

Таблица projects - информация о проектах:

- project_id - уникальный идентификатор проекта; 
- employee_id - уникальный идентификатор сотрудника, который работает над проектом;
- project_name - название проекта;
- hours_worked - количество часов, которое данный сотрудник работает над данным проектом.

Вывести id сотрудника, имя сотрудника и имя его руководителя для **каждого** сотрудника

In [ ]:
SELECT e.employee_id,
       e.name,
       em.name as manager_name
  FROM employees as e
  LEFT JOIN employees as em
       ON em.employee_id = e.manager_id
 ORDER BY employee_id

Вывести общую информацию о проектах: список всех проектов, общее количество часов потраченных на проекты и колчество сотрудников, которые работают над проектом

In [ ]:
SELECT project_name,
       sum (hours_worked) as total_hour,
       count(distinct employee_id) as emp_count
  FROM projects
 GROUP BY project_name
 ORDER BY project_name

Вывести топ-3 сотрудников, которые провели больше всего часов, работая над проектами

In [ ]:
SELECT e.name,
       sum(p.hours_worked) as total_hour
  FROM projects as p join employees as e
        ON e.employee_id = p.employee_id
 GROUP BY e.name
 ORDER BY 2 DESC 
 LIMIT 3

### Задание 3

Таблица peas - информация о решенных задачах:

- st_id - уникальный идентификатор ученика;
- timest - время решения задания;
- correct - правильно ли решена задача;
- subject - дисциплина, в которой находится задача.

Таблица studs - информация о студентах:

- st_id - уникальный идентификатор ученика;
- test_grp - к какой группе принадлежит студент.

Таблица final_project_check - информация о покупках студентов:

- st_id - уникальный идентификатор ученика;
- sale_time - время покупки;
- money - цена покупки;
- subject - дисциплина.

#### Очень усердные ученики.

#### Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

#### Решение

In [ ]:
SELECT COUNT(1) AS diligent_students
  FROM (SELECT st_id
          FROM peas
         WHERE correct = true
         GROUP BY st_id
        HAVING COUNT(1) >= 20) AS DS

#### Оптимизация воронки

#### Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU 
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

#### Решение

In [ ]:
WITH
  -- 1. Определяем всех пользователей в эксперименте с их группами
  experiment_users AS (
    SELECT DISTINCT st_id, test_grp
      FROM studs
  ),

  -- 2. Определяем пользователей, которые являются активными (> 10 правильно решенных горошин)
  active_users AS (
    SELECT st_id
      FROM peas
     WHERE correct = TRUE
     GROUP BY st_id
    HAVING COUNT(*) > 10
  ),

  -- 3. Определяем пользователей, которые являются активными по математике (>= 2 правильно решенных задач по математике)
  math_active_users AS (
    SELECT st_id
      FROM peas
     WHERE correct = TRUE AND subject = 'Math'
     GROUP BY st_id
    HAVING COUNT(*) >= 2
  ),

  -- 4. Берем все поля из таблицы со студентами и их покупками и переименовываем ее
  purchases AS (
    SELECT st_id, subject, sale_time, money
      FROM final_project_check
  ),

  -- 5. Считаем общую выручку по группам
  total_revenue AS (
    SELECT u.test_grp, SUM(money) AS total_revenue
      FROM experiment_users u
      LEFT JOIN purchases p ON u.st_id = p.st_id
     GROUP BY u.test_grp
  ),

  -- 6. Считаем количество покупок
  purchase_counts AS (
      SELECT test_grp, COUNT(DISTINCT st_id) as num_purchases
        FROM (SELECT DISTINCT u.test_grp, p.st_id
                FROM experiment_users u
                JOIN purchases p ON u.st_id = p.st_id) as sub
       GROUP BY test_grp
  ),

  -- 7. Считаем количество покупок по математике
  math_purchase_counts AS (
      SELECT test_grp, COUNT(DISTINCT st_id) as num_math_purchases
        FROM (SELECT DISTINCT u.test_grp, p.st_id
                FROM experiment_users u
                JOIN purchases p ON u.st_id = p.st_id AND p.subject = 'Math'
                JOIN math_active_users mau ON mau.st_id = u.st_id ) as sub
       GROUP BY test_grp
  ),

  -- 8. Считаем количество активных пользователей по группам
  active_user_counts AS (
    SELECT u.test_grp, COUNT(DISTINCT a.st_id) AS num_active_users
      FROM experiment_users u
      LEFT JOIN active_users a ON u.st_id = a.st_id
     GROUP BY u.test_grp
  ),

  -- 9. Считаем общее количество пользователей по группам
  total_user_counts AS (
    SELECT test_grp, COUNT(DISTINCT st_id) AS total_users
      FROM experiment_users
     GROUP BY test_grp
  ),

  -- 10. Считаем количество активных пользователей по математике по группам
  math_active_user_counts AS (
    SELECT u.test_grp, COUNT(DISTINCT mau.st_id) AS num_math_active_users
      FROM experiment_users u
      LEFT JOIN math_active_users mau ON u.st_id = mau.st_id
     GROUP BY u.test_grp
  )

-- 11. Собираем все вместе
SELECT
  t.test_grp,
  tr.total_revenue::numeric / t.total_users AS "ARPU",
  tr.total_revenue::numeric / COALESCE(auc.num_active_users, 1) AS "ARPAU",  -- Используем COALESCE для избежания ошибки деления на 0
  ROUND(pc.num_purchases::numeric / t.total_users * 100, 2) AS "CR_to_purchase",
  ROUND(pc.num_purchases::numeric / COALESCE(auc.num_active_users, 1) * 100, 2) AS "CR_active_to_purchase", 
  ROUND(COALESCE(mpc.num_math_purchases::numeric / mauc.num_math_active_users, 0) * 100, 2) AS "CR_math_active_to_math_purchase"
  FROM total_user_counts t
  JOIN total_revenue tr ON t.test_grp = tr.test_grp
  LEFT JOIN active_user_counts auc ON t.test_grp = auc.test_grp
  LEFT JOIN math_active_user_counts mauc ON t.test_grp = mauc.test_grp
  LEFT JOIN purchase_counts pc ON t.test_grp = pc.test_grp
  LEFT JOIN math_purchase_counts mpc ON t.test_grp = mpc.test_grp
 ORDER BY t.test_grp